In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Configuration
INPUT_SIZE = 3  # thickness, height, angle (what we want to generate)
CONDITION_SIZE = 1  # bending_stiffness (what we condition on)
HIDDEN_DIM = 64
LATENT_DIM = 8
LEARNING_RATE = 0.001
BATCH_SIZE = 32
NUM_EPOCHS = 100
TEST_SIZE = 0.2
RANDOM_STATE = 42
device = "cpu"

class ConditionalVAE(nn.Module):
    """
    Conditional VAE that generates [thickness, height, angle] 
    given a target bending_stiffness value
    """
    def __init__(self, input_dim=3, condition_dim=1, hidden_dim=64, latent_dim=8):
        super(ConditionalVAE, self).__init__()
        
        # Encoder: takes input + condition
        self.encoder = nn.Sequential(
            nn.Linear(input_dim + condition_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU()
        )
        
        # Latent space
        self.mean_layer = nn.Linear(hidden_dim, latent_dim)
        self.logvar_layer = nn.Linear(hidden_dim, latent_dim)
        
        # Decoder: takes latent + condition
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim + condition_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim)
        )
     
    def encode(self, x, condition):
        # Concatenate input with condition
        x_cond = torch.cat([x, condition], dim=1)
        h = self.encoder(x_cond)
        mean = self.mean_layer(h)
        logvar = self.logvar_layer(h)
        return mean, logvar
    
    def reparameterize(self, mean, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mean + eps * std
    
    def decode(self, z, condition):
        # Concatenate latent with condition
        z_cond = torch.cat([z, condition], dim=1)
        return self.decoder(z_cond)
    
    def forward(self, x, condition):
        mean, logvar = self.encode(x, condition)
        z = self.reparameterize(mean, logvar)
        x_hat = self.decode(z, condition)
        return x_hat, mean, logvar

def loss_function(x, x_hat, mean, log_var):
    """VAE loss = Reconstruction loss + KL divergence"""
    reconstruction_loss = nn.functional.mse_loss(x_hat, x, reduction='sum')
    kld = -0.5 * torch.sum(1 + log_var - mean.pow(2) - log_var.exp())
    return reconstruction_loss + kld

def train_cvae(model, optimizer, train_loader, epochs, device):
    """Train the conditional VAE"""
    model.train()
    for epoch in range(epochs):
        overall_loss = 0
        total_samples = 0
        
        for batch_idx, (x, condition) in enumerate(train_loader):
            x = x.to(device)
            condition = condition.to(device)
            current_batch_size = x.size(0)
            
            optimizer.zero_grad()
            
            # Forward pass
            x_hat, mean, log_var = model(x, condition)
            loss = loss_function(x, x_hat, mean, log_var)
            
            overall_loss += loss.item()
            total_samples += current_batch_size
            
            # Backward pass
            loss.backward()
            optimizer.step()
        
        avg_loss = overall_loss / total_samples
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch + 1}/{epochs}\tAverage Loss: {avg_loss:.6f}")
    
    return model

def generate_designs(model, target_stiffness, scaler_X, scaler_y, 
                     n_samples=10, device='cpu'):
    """
    Generate designs given a target bending stiffness
    
    Args:
        model: Trained CVAE model
        target_stiffness: Desired bending stiffness value
        scaler_X: StandardScaler fitted on X (thickness, height, angle)
        scaler_y: StandardScaler fitted on y (bending_stiffness)
        n_samples: Number of design variations to generate
        device: 'cpu' or 'cuda'
    
    Returns:
        numpy array of shape (n_samples, 3) with [thickness, height, angle]
    """
    model.eval()
    
    with torch.no_grad():
        # Normalize the target stiffness
        target_normalized = scaler_y.transform([[target_stiffness]])
        target_tensor = torch.FloatTensor(target_normalized).to(device)
        
        # Repeat for n_samples
        condition = target_tensor.repeat(n_samples, 1)
        
        # Sample from prior distribution
        z = torch.randn(n_samples, model.mean_layer.out_features).to(device)
        
        # Generate designs
        generated = model.decode(z, condition)
        
        # Convert back to numpy and denormalize
        generated_np = generated.cpu().numpy()
        designs = scaler_X.inverse_transform(generated_np)
        
    return designs

# Example usage:
if __name__ == "__main__":
    # Load and prepare data
    df = pd.read_csv(r'../MLP/processed_bending_stiffness.csv')
    df = df.drop_duplicates()
    
    X = df[['Thickness', 'Height', 'Angle (deg)']].values
    y = df['Bending_Stiffness'].values.reshape(-1, 1)
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE
    )
    
    # Normalize data (IMPORTANT for VAE training!)
    scaler_X = StandardScaler()
    scaler_y = StandardScaler()
    
    X_train_scaled = scaler_X.fit_transform(X_train)
    y_train_scaled = scaler_y.fit_transform(y_train)
    X_test_scaled = scaler_X.transform(X_test)
    y_test_scaled = scaler_y.transform(y_test)
    
    # Create DataLoaders
    X_train_tensor = torch.FloatTensor(X_train_scaled)
    y_train_tensor = torch.FloatTensor(y_train_scaled)
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, 
                            shuffle=True, drop_last=False)
    
    # Initialize and train model
    model = ConditionalVAE(
        input_dim=INPUT_SIZE,
        condition_dim=CONDITION_SIZE,
        hidden_dim=HIDDEN_DIM,
        latent_dim=LATENT_DIM
    ).to(device)
    
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    
    print("Training Conditional VAE...")
    model = train_cvae(model, optimizer, train_loader, NUM_EPOCHS, device)
    
    # Generate designs for a target stiffness
    print("\n" + "="*60)
    print("GENERATING DESIGNS")
    print("="*60)
    
    target_stiffness = 50.0  # Example target
    designs = generate_designs(
        model, 
        target_stiffness, 
        scaler_X, 
        scaler_y, 
        n_samples=10, 
        device=device
    )
    
    print(f"\nTarget Bending Stiffness: {target_stiffness}")
    print("\nGenerated Designs:")
    print("-" * 60)
    print(f"{'#':<5} {'Thickness':<15} {'Height':<15} {'Angle (deg)':<15}")
    print("-" * 60)
    for i, design in enumerate(designs, 1):
        print(f"{i:<5} {design[0]:<15.4f} {design[1]:<15.4f} {design[2]:<15.4f}")

Training Conditional VAE...
Epoch 10/100	Average Loss: 1.524517
Epoch 20/100	Average Loss: 1.437921
Epoch 30/100	Average Loss: 1.372953
Epoch 40/100	Average Loss: 1.347630
Epoch 50/100	Average Loss: 1.345514
Epoch 60/100	Average Loss: 1.345271
Epoch 70/100	Average Loss: 1.315832
Epoch 80/100	Average Loss: 1.295861
Epoch 90/100	Average Loss: 1.306209
Epoch 100/100	Average Loss: 1.314967

GENERATING DESIGNS

Target Bending Stiffness: 50.0

Generated Designs:
------------------------------------------------------------
#     Thickness       Height          Angle (deg)    
------------------------------------------------------------
1     5.6731          19.2298         48.6431        
2     4.3776          27.0395         50.2104        
3     5.7778          21.9993         59.7563        
4     4.2209          29.9346         36.5709        
5     7.0927          17.1171         38.3083        
6     5.1438          25.0534         34.9352        
7     7.4318          17.9710         3